In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated_umap.csv"
).resolve()
# shiny output path
shiny_output_path = pathlib.Path(
    "../temporal_shiny_app/CLS_features_annotated_umap.csv"
).resolve()

output_path.parent.mkdir(parents=True, exist_ok=True)
shiny_output_path.parent.mkdir(parents=True, exist_ok=True)

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,channel488-1_cls_feature_0,...,channel_DNA_cls_feature_90,channel_DNA_cls_feature_91,channel_DNA_cls_feature_92,channel_DNA_cls_feature_93,channel_DNA_cls_feature_94,channel_DNA_cls_feature_95,channel_DNA_cls_feature_96,channel_DNA_cls_feature_97,channel_DNA_cls_feature_98,channel_DNA_cls_feature_99
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,-0.012062,...,0.034462,-0.005022,-0.032672,0.056519,-0.081751,0.035719,0.008650,0.017762,0.038345,-0.015309
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,-0.014538,...,0.008132,-0.048781,-0.029867,0.013783,-0.089631,0.029462,0.013753,-0.041825,0.045902,-0.025322
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,-0.010044,...,0.016777,-0.022970,-0.012937,0.031086,-0.063206,0.017341,0.006725,0.058906,0.051218,-0.021713
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,0.021043,...,0.016239,0.026569,0.000094,0.040027,-0.120004,0.012140,0.008501,0.048850,0.048612,0.017343
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,-0.009109,...,-0.009832,-0.015505,-0.035550,0.037506,-0.102835,-0.030543,0.026912,-0.000418,0.092182,-0.001539


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (182804, 9)
feature_df shape: (182804, 1536)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniforge3/envs/scDINO_env/lib/python3.8/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,UMAP1,UMAP2
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,1.796733,3.370771
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,2.328630,2.666051
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,3.814675,2.137078
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,2.172429,3.406138
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,cell,Staurosporine,0.0,negative,-1.038915,0.446976


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)

# save the UMAP embeddings to shiny app
umap_embedding_df.to_csv(shiny_output_path)